In [ ]:
!pip install praw pandas
!pip install pandas requests
!pip install -q praw
import os
import time
import requests
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine


# Load environment variables from .env file
load_dotenv()


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python3 -m pip install --upgrade pip


True

In [2]:
# Retrieve credentials from environment variables
pg_user = os.getenv('PG_USER')
pg_password = os.getenv('PG_PASSWORD')
pg_host = os.getenv('PG_HOST')
pg_db = os.getenv('PG_DB')

# Build the connection string
conn_str = f"postgresql+psycopg2://{pg_user}:{pg_password}@{pg_host}/{pg_db}"
engine = create_engine(conn_str)

In [3]:
import requests
import csv
import time

def fetch_all_threads(subreddit, max_pages=10, delay=2):
    headers = {'User-Agent': 'RedditScraper/1.0'}
    base_url = f'https://www.reddit.com/r/formula1/.json'
    after = None
    all_posts = []

    for _ in range(max_pages):
        url = base_url + (f'?after={after}' if after else '')
        response = requests.get(url, headers=headers)

        if response.status_code != 200:
            print(f"Failed to fetch data: {response.status_code}")
            break

        data = response.json()
        posts = data['data']['children']
        all_posts.extend([post['data'] for post in posts])

        after = data['data'].get('after')
        if not after:
            break

        time.sleep(delay)

    return all_posts

def write_to_csv(posts, filename):
    with open(filename, mode='w', newline='', encoding='utf-8') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=['title', 'selftext'])
        writer.writeheader()

        for post in posts:
            writer.writerow({
                'title': post.get('title', '').replace('\n', ' ').strip(),
                'selftext': post.get('selftext', '').replace('\n', ' ').strip()
            })

# Usage
subreddit = 'formula1'
threads = fetch_all_threads(subreddit, max_pages=5)
csv_filename = f'{subreddit}_threads.csv'

write_to_csv(threads, csv_filename)
print(f"Wrote {len(threads)} threads to {csv_filename}")

Failed to fetch data: 403
Wrote 0 threads to formula1_threads.csv


In [4]:
# Convert threads (list of dicts) into a DataFrame
df_web_scraped = pd.DataFrame(threads)

In [5]:
df_web_scrape = pd.read_csv('formula1_threads.csv')
print(df_web_scrape.shape)
print(df_web_scrape.head())

(127, 2)
                                               title  \
0          2025 Saudi Arabian GP - Day After Debrief   
1  Ask r/Formula1 Anything - Daily Discussion Thread   
2  A small detail I noticed on the special Miami ...   
3                       The Ferrari livery for Miami   
4      Remember how good the Ferraris looked pre-HP?   

                                            selftext  
0  Welcome to the Day after Debrief discussion th...  
1  # Welcome to the [r/formula1](https://www.redd...  
2                                                NaN  
3                                Consider me whelmed  
4                                                NaN  


In [13]:
import pandas as pd

# Read CSV into DataFrame (this file was created earlier by your Reddit script)
df_web_scrape = pd.read_csv('formula1_threads.csv')

# Write to PostgreSQL in 'raw' schema
df_web_scrape.to_sql(
    name='web_scrape_formula1',
    con=engine,
    schema='raw',
    if_exists='replace',
    index=False
)

print("Web scrape CSV uploaded to PostgreSQL successfully.")

Web scrape CSV uploaded to PostgreSQL successfully.
